In [3]:
import pandas as pd
import numpy as np

# This notebook is old junk analysis

In [4]:
path = 'data/CDCdata/COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv'
chunksize=250000

## What are the possible values of current_status from testing data?
* We see below there are only two values -- laboratory confirmed and probable -- and no missing values.
* Probable status is only 10% of cases.

**Thus we consider all records from this dataset to be positive cases for Covid 19**

In [ ]:
# What are the possible statuses of test?
raw_iter = pd.read_csv(path, chunksize=chunksize)

status_counts = pd.Series(dtype='float')
for chunk in raw_iter:
    status_counts = status_counts.combine(chunk.current_status.value_counts(), np.add, fill_value=0)

status_counts

# laboratory confirmed and probable
# We also assume current_status means test result

In [38]:
# Are there any missing test results?
raw_iter = pd.read_csv(path, chunksize=chunksize)

nan_count = 0
for chunk in raw_iter:
    nan_count = chunk['current_status'].isna().sum()

nan_count

# No

0

## What data will we collect from each chunk?

In [116]:
# Look deeper into a single chunk
chunk = next(raw_iter)

In [53]:
chunk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 19 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   case_month                       250000 non-null  object 
 1   res_state                        250000 non-null  object 
 2   state_fips_code                  250000 non-null  int64  
 3   res_county                       249182 non-null  object 
 4   county_fips_code                 249182 non-null  float64
 5   age_group                        237646 non-null  object 
 6   sex                              208438 non-null  object 
 7   race                             136223 non-null  object 
 8   ethnicity                        117317 non-null  object 
 9   case_positive_specimen_interval  81345 non-null   float64
 10  case_onset_interval              123475 non-null  float64
 11  process                          250000 non-null  object 
 12  ex

In [42]:
chunk

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
0,2020-12,MN,27,DODGE,27039.0,18 to 49 years,NaN,NaN,NaN,-1.0,0.0,Missing,Missing,Laboratory-confirmed case,Symptomatic,Missing,Missing,No,NaN
1,2020-11,MI,26,GLADWIN,26051.0,18 to 49 years,NaN,NaN,NaN,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Unknown,NaN
2,2021-04,VT,50,ORLEANS,50019.0,18 to 49 years,NaN,NaN,NaN,0.0,0.0,Multiple,Yes,Laboratory-confirmed case,Symptomatic,No,Missing,NaN,Yes
3,2020-11,AL,1,LIMESTONE,1083.0,0 - 17 years,Female,Missing,Missing,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN
4,2021-01,NY,36,SCHOHARIE,36095.0,0 - 17 years,Female,NaN,NaN,0.0,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,2020-12,KY,21,BULLITT,21029.0,65+ years,Female,White,Non-Hispanic/Latino,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,No,NaN
249996,2020-12,WI,55,GREEN,55045.0,65+ years,Female,White,Non-Hispanic/Latino,NaN,0.0,Missing,Missing,Laboratory-confirmed case,Symptomatic,Yes,Missing,NaN,NaN
249997,2021-01,NY,36,ONTARIO,36069.0,65+ years,Female,White,Non-Hispanic/Latino,0.0,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,No,NaN
249998,2020-10,MT,30,SILVER BOW,30093.0,65+ years,Female,White,Non-Hispanic/Latino,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Unknown,Missing,NaN,NaN


In [1]:
# Oops, lot of 'Missing' values... meaning NaN, right? Also, FIPS should be a string.
chunk = chunk.replace('Missing', np.nan)

# And let's fix month column too
chunk['case_month'] = pd.to_datetime(chunk['case_month'], format='%Y-%m').dt.to_period('M')
chunk.info()

# It's tempting to use the hops, icu, and death statistics, but it seem half that data wasn't recorded, so we ignore.

NameError: name 'chunk' is not defined

In [74]:
# We presumedly cannot use the 800+ cases with missing counties (as each needs a county to find local hospitalizations).
# Sanity check though: How spread out are they (are they concentrated in a particular state/month, e.g.)?
chunk[chunk['county_fips_code'].isna()].groupby(['case_month'])['res_state'].value_counts().sort_index()

# Some states seem a little more negiligent than others, but thery are spread out on the calendar pretty well

case_month  res_state
2020-03-01  IN           11
2020-05-01  AZ           25
            MD           21
2020-06-01  AZ           12
2020-07-01  AZ           12
            GA           66
            IN           28
2020-09-01  IL           13
            PA           11
2020-10-01  IN           41
            MD           18
            PA           49
2020-11-01  AZ           36
            GA           16
            IL           26
            IN           11
            MD           10
2020-12-01  AZ           13
            IL           43
            IN           54
            PA           14
2021-01-01  AZ           12
            IN           19
            MD           29
            PA           38
2021-02-01  AZ           23
            MD           18
            PA           12
2021-03-01  AZ           16
            MD           18
2021-04-01  MD           49
            PA           21
2021-05-01  PA           18
2021-07-01  AZ           15
Name: res_state, dtype: in

In [118]:
# We basically want to capture the # of cases within each county during each unit of time
# Specifically, county FIPS, case_month, case count

# We'll just drop cases with NaN county codes (below)
dropped_cases = chunk[chunk['county_fips_code'].isna()] # Save these for later
chunk = chunk.drop(dropped_cases.index)

# Convert county_fips_code to a string
chunk['county_fips_code'] = chunk['county_fips_code'].astype(int).astype(str).str.zfill(5)

# Create cases table
cases = chunk.groupby(['county_fips_code', 'case_month'])['current_status'].count()
cases.index = cases.index.rename(['FIPS', 'month'])
cases.name = 'case_count'
cases

FIPS   month     
01001  2020-09-01    14
       2020-10-01    20
       2020-11-01    56
       2021-01-01    17
       2021-02-01    15
                     ..
55141  2020-10-01    14
       2020-11-01    36
       2020-12-01    11
       2021-01-01    71
       2021-02-01    13
Name: case_count, Length: 9231, dtype: int64

## Now to aggregate all the chunks

In [119]:
def process_chunk(chunk):
    chunk = chunk.replace('Missing', np.nan)
    chunk['case_month'] = pd.to_datetime(chunk['case_month'], format='%Y-%m')
    dropped_chunk = chunk[chunk['county_fips_code'].isna()]
    chunk = chunk.drop(dropped_chunk.index)
    chunk['county_fips_code'] = chunk['county_fips_code'].astype(int).astype(str).str.zfill(5)
    
    cases = chunk.groupby(['county_fips_code', 'case_month'])['current_status'].count()
    cases.index = cases.index.rename(['FIPS', 'month'])
    cases.name = 'case_count'
    
    dropped_cases = dropped_chunk.groupby(['res_state', 'case_month'])['current_status'].count()
    dropped_cases.index = dropped_cases.index.rename(['state', 'month'])
    dropped_cases.name = 'case_count'

    return cases, dropped_cases

In [146]:
# Reset the raw file
chunksize = 250000
raw_iter2 = pd.read_csv(path, chunksize=chunksize)

In [147]:
cases = pd.DataFrame({'case_count': []}, index=pd.MultiIndex.from_arrays([[],[]], names=('FIPS', 'month')))
dropped_cases = pd.DataFrame({'case_count': []}, index=pd.MultiIndex.from_arrays([[],[]], names=('state', 'month')))

for chunk in raw_iter2:
    new_cass, new_drpd_cass = process_chunk(chunk)
    
    cases = pd.merge(cases, new_cass, how='outer', on=['FIPS', 'month']).fillna(0.0, downcast='infer')
    cases['case_count'] = cases['case_count_x'] + cases['case_count_y']
    cases = cases.drop(['case_count_x', 'case_count_y'], axis=1)
    
    dropped_cases = pd.merge(dropped_cases, new_drpd_cass, how='outer', on=['state', 'month']).fillna(0.0, downcast='infer')
    dropped_cases['case_count'] = dropped_cases['case_count_x'] + dropped_cases['case_count_y']
    dropped_cases = dropped_cases.drop(['case_count_x', 'case_count_y'], axis=1)

C:\Users\Michael\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [148]:
cases

case_count
FIPS  month                 
01001 2020-09-01         331
      2020-10-01         393
      2020-11-01         733
      2021-01-01        1499
      2021-02-01         562
...                      ...
32003 2020-01-01          57
36081 2020-01-01          15
06071 2020-01-01          49
04013 2020-01-01          12
17031 2020-01-01          23

[24042 rows x 1 columns]

In [149]:
cases.sum()

case_count    26327514
dtype: int64

In [161]:
print('# of US cases:', cases.sum()[0])
print('# of US counties reporting cases:', cases.reset_index()['FIPS'].nunique())

# Various new sources report the US has had 37M+ cases, so my source (with 26M+ cases) is likely lacking.
# Furthermore, only 1530 out of 3108 counties in the US reported any cases to my source.

# I think we only pursue hospitalization rates in the counties we feel confident about (if there are any!).

# of US cases: 26327514
# of US counties reporting cases: 1530


## Further investigations follow, not essential

In [150]:
cases.reset_index().groupby('FIPS')['month'].count().value_counts()

17    675
16    282
15    191
14    109
18     68
13     62
12     35
19     34
11     20
6      11
7      11
9       7
8       7
10      5
1       4
4       3
5       2
3       2
2       2
Name: month, dtype: int64

In [151]:
dropped_cases

case_count
state month                 
AZ    2020-05-01         146
      2020-06-01         364
      2020-07-01         220
      2020-11-01         441
      2020-12-01         773
...                      ...
FL    2021-04-01         486
      2021-05-01         293
      2021-06-01         387
      2021-07-01         365
GA    2020-02-01         101

[943 rows x 1 columns]

In [152]:
dropped_cases.sum()

case_count    1550852
dtype: int64

In [153]:
dropped_cases.reset_index().groupby('month')['case_count'].sum().sort_index()

month
2020-01-01      2146
2020-02-01      1842
2020-03-01     18959
2020-04-01     44761
2020-05-01     50107
2020-06-01     58264
2020-07-01    101718
2020-08-01     82274
2020-09-01     85540
2020-10-01    144120
2020-11-01    239285
2020-12-01    238800
2021-01-01    204725
2021-02-01     76228
2021-03-01     60453
2021-04-01     57532
2021-05-01     34679
2021-06-01     25311
2021-07-01     24108
Name: case_count, dtype: int64